In [ ]:
import os
import sys
sys.path.insert(0, "../..")
sys.path.insert(0, "../../../acquire")
import ipywidgets as widgets

from HUGS.Processing import search

from Acquire.Client import User, Drive, Service, PAR, Authorisation, StorageCreds

In [ ]:
# Autoreload modules before executing code, useful during development
%load_ext autoreload
%autoreload 2

## Start with uploading and processing of the data

In [ ]:
base_url = "https://hugs.acquire-aaai.com/t"
hugs_url = "https://hugs.acquire-aaai.com"
user = User(username="gareth", identity_url="%s/identity" % base_url)
response = user.request_login()

Login by visiting: https://login.acquire-aaai.com?id=a0-a6/2a.42.db.f9
(please check that this page displays the message 'fluffy mice climb quietly')


Check we're logged in

In [ ]:
user.wait_for_login()

True

In [ ]:
filename = "bsd.picarro.1minute.248m.dat"
dir_path = os.path.abspath("")
test_data = "../../test/data/proc_test_data/CRDS"
filepath = os.path.join(dir_path, test_data, filename)

hugs = Service(service_url="%s/hugs" % base_url)

creds = StorageCreds(user=user, service_url="%s/storage" % base_url)
drive = Drive(creds=creds, name="test_drive")
filemeta = drive.upload(filepath)
par = PAR(location=filemeta.location(), user=user)

par_secret = hugs.encrypt_data(par.secret())
auth = Authorisation(resource="process", user=user)

In [ ]:
def upload_file(authenticated_user, filepath):
    """ Upload a file to the object store
    
        Args:
            filepath (str): Path to file to upload
        Returns:
            None    
    """
    creds = StorageCreds(user=authenticated_user, service_url="%s/storage" % base_url)
    drive = Drive(creds=creds, name="test_drive")
    filemeta = drive.upload(filepath)
    par = PAR(location=filemeta.location(), user=user)

    par_secret = hugs.encrypt_data(par.secret())
    auth = Authorisation(resource="process", user=user)

In [ ]:
def get_CRDS_path(filename):
    dir_path = os.path.abspath("")
    test_data = "../../test/data/proc_test_data/CRDS"
    return os.path.join(dir_path, test_data, filename)

### Upload files for processing


Clear the Datasources from CRDS and GC objects (currently Datasource UUIDs are just being randomly generated and we get multiple Datasources holding the same data)

In [ ]:
response = hugs.call_function(function="clear_datasources", args={})
response

In [ ]:
from HUGS.Client import Process

filename = "bsd.picarro.1minute.248m.dat"
filepath = get_CRDS_path(filename)

processing = Process(service_url=base_url)
result = processing.process_files(user=user, files=filepath, data_type="CRDS")
print(result)

{'bsd.picarro.1minute.248m.dat': ['dc01098c-07e1-4e86-9019-d830feec52c3', '68daa306-8a7a-47d2-b8fb-d63513cd1fc4', '98f99b5b-1f11-4794-8607-13eec12552d2']}


Upload some Heathfield data

In [ ]:
filename = "hfd.picarro.1minute.100m_min.dat"
filepath = get_CRDS_path(filename)
result = processing.process_files(user=user, files=filepath, data_type="CRDS")
print(result)

{'hfd.picarro.1minute.100m_min.dat': ['d277117f-65d7-4e56-955b-dff04f8e2da9', '19640040-8f07-4642-a9a6-2a903e02578e', '74055929-beff-4b53-a15a-4ffa837bebfa']}


In [ ]:
from HUGS.Processing import search

search_results = None

search_terms = widgets.Text(
                    value="",
                    placeholder = "Search",
                    description = "Search terms:",
                    disabled = False
)

locations = widgets.Text(
                value="",
                placeholder = "BSD, HFD",
                description = "Locations:",
                disabled = False
)

data_type = widgets.Dropdown(
                options=["CRDS", "GC"], 
                value="CRDS",
                description="Data type",
                disabled=False
    )



layout=widgets.Layout(display = "flex", width = "50%")
search_button = widgets.Button(description="Search", button_style="success", \
                               layout=widgets.Layout(flex='1 1 0%', width='25%'))

start_picker = widgets.DatePicker(
                    description='Start date',
                    disabled=False
)

end_picker = widgets.DatePicker(
                    description='End date',
                    disabled=False
)


box = widgets.VBox(children=[search_terms, locations, start_picker, end_picker, data_type,
                                 search_button], layout=layout)



def call_search(x):
    """ Call the search function and pass it the values 
        in the text boxes
            
    """
    from datetime import datetime
    from Acquire.ObjectStore import datetime_to_string
    from HUGS.Client import Search
    start = datetime(1970, 1,1) # datetime.combine(start_picker.value, datetime.min.time())
    end = datetime.now() # datetime.combine(end_picker.value, datetime.min.time())
    
    split_search_terms = search_terms.value.replace(" ", "").split(",")
    split_locations = locations.value.replace(" ", "").split(",")
    
    global search_results
    search = Search(service_url=base_url)
    search_results = search.search(search_terms=split_search_terms, locations=split_locations, data_type=data_type.value, start_datetime=start, end_datetime=end)

search_button.on_click(call_search)




In [ ]:
box

['co2', 'co'] ['bsd', 'hfd']


In [ ]:
print(search_results)

{'bsd_co2': ['data/uuid/68daa306-8a7a-47d2-b8fb-d63513cd1fc4/2014-01-30T10:52:30_2014-01-30T14:20:30'], 'hfd_co2': ['data/uuid/19640040-8f07-4642-a9a6-2a903e02578e/2013-12-04T14:02:30_2013-12-25T22:56:30', 'data/uuid/19640040-8f07-4642-a9a6-2a903e02578e/2014-01-01T18:25:30_2014-12-28T08:02:30', 'data/uuid/19640040-8f07-4642-a9a6-2a903e02578e/2015-01-04T17:22:30_2015-12-30T14:55:30', 'data/uuid/19640040-8f07-4642-a9a6-2a903e02578e/2016-01-06T18:01:30_2016-12-29T02:17:30', 'data/uuid/19640040-8f07-4642-a9a6-2a903e02578e/2017-01-01T19:56:30_2017-12-26T07:35:30', 'data/uuid/19640040-8f07-4642-a9a6-2a903e02578e/2018-01-02T13:10:30_2018-12-30T10:12:30', 'data/uuid/19640040-8f07-4642-a9a6-2a903e02578e/2019-01-06T11:02:30_2019-05-21T15:46:30'], 'bsd_co': ['data/uuid/98f99b5b-1f11-4794-8607-13eec12552d2/2014-01-30T10:52:30_2014-01-30T14:20:30'], 'hfd_co': ['data/uuid/74055929-beff-4b53-a15a-4ffa837bebfa/2013-12-04T14:02:30_2013-12-25T22:56:30', 'data/uuid/74055929-beff-4b53-a15a-4ffa837bebfa/20

Now take the result dictionary, parse the dictionary into a blocks that can be selected by the user

In [ ]:
def parse_results(results):
    """ Split the keys into a list of each key and the date that the data covers
        
        Args:
            results (dict): Dictionary of search results
        Returns:
            list (tuple): List of date, data key list pairs
    """
    date_keys = {}
    for key in results.keys():
        keys = sorted(results[key])
        start_key = keys[0]
        end_key = keys[-1]
        # Get the first and last dates from the keys in the search results
        start_date = start_key.split("/")[-1].split("_")[0]
        end_date = end_key.split("/")[-1].split("_")[-1]
        
        dates_covered = start_date + "_" + end_date
        
        date_keys[key] = {"dates": dates_covered, "keys": keys}
        
    return date_keys

Site, gas, dates and checkbox

In [ ]:
date_keys = parse_results(search_results)
print(date_keys)

{'bsd_co2': {'dates': '2014-01-30T10:52:30_2014-01-30T14:20:30', 'keys': ['data/uuid/68daa306-8a7a-47d2-b8fb-d63513cd1fc4/2014-01-30T10:52:30_2014-01-30T14:20:30']}, 'hfd_co2': {'dates': '2013-12-04T14:02:30_2019-05-21T15:46:30', 'keys': ['data/uuid/19640040-8f07-4642-a9a6-2a903e02578e/2013-12-04T14:02:30_2013-12-25T22:56:30', 'data/uuid/19640040-8f07-4642-a9a6-2a903e02578e/2014-01-01T18:25:30_2014-12-28T08:02:30', 'data/uuid/19640040-8f07-4642-a9a6-2a903e02578e/2015-01-04T17:22:30_2015-12-30T14:55:30', 'data/uuid/19640040-8f07-4642-a9a6-2a903e02578e/2016-01-06T18:01:30_2016-12-29T02:17:30', 'data/uuid/19640040-8f07-4642-a9a6-2a903e02578e/2017-01-01T19:56:30_2017-12-26T07:35:30', 'data/uuid/19640040-8f07-4642-a9a6-2a903e02578e/2018-01-02T13:10:30_2018-12-30T10:12:30', 'data/uuid/19640040-8f07-4642-a9a6-2a903e02578e/2019-01-06T11:02:30_2019-05-21T15:46:30']}, 'bsd_co': {'dates': '2014-01-30T10:52:30_2014-01-30T14:20:30', 'keys': ['data/uuid/98f99b5b-1f11-4794-8607-13eec12552d2/2014-01-3

In [ ]:
from ipywidgets import Button, ButtonStyle, Checkbox, HTML, Label, HBox, VBox

table_style = {'description_width': 'initial'}
table_layout = {'width':'100px', 'min_width':'100px', 'height':'28px', 'min_height':'28px'}
date_layout = {'width':'275px', 'min_width':'200px', 'height':'28px', 'min_height':'28px'}
checkbox_layout = {'width':'100px', 'min_width':'100px', 'height':'28px', 'min_height':'28px'}
statusbar_layout = {'width':'250px', 'min_width':'250px', 'height':'28px', 'min_height':'28px'}

# row_layout = {'width':'200px', 'min_width':'200px'}

header_label_site = HTML(value=f"<b>Site</b>", layout=table_layout)
header_label_gas = HTML(value=f"<b>Gas</b>", layout=table_layout)
header_label_dates = HTML(value=f"<b>Dates</b>", layout=date_layout)
header_label_select = HTML(value=f"<b>Select</b>", layout=checkbox_layout)

checkbox_objects = []
search_keys = []

site_labels = []
date_labels = []
gas_labels = []
for key in date_keys:
    # Create the checkboxes
    checkbox = Checkbox(value=False)
    checkbox_objects.append(checkbox)
    search_keys.append(key)
    
    dates = date_keys[key]["dates"].replace("_", " to ").replace("T", " ")
    date_label = Label(value=dates, layout=date_layout)
    
    split_key = key.split("_")
    site_name = split_key[0].upper()
    gas_name = split_key[1].upper()
    
    gas_label = Label(value=gas_name, layout=table_layout)
    site_label = Label(value=site_name, layout=table_layout)

    date_labels.append(date_label)
    site_labels.append(site_label)
    gas_labels.append(gas_label)

    
arg_dict = {search_keys[i]: checkbox for i, checkbox in enumerate(checkbox_objects)}

header_box = HBox(children=[header_label_site, header_label_gas, header_label_dates, header_label_select])

site_vbox = VBox(children=site_labels)
gas_vbox = VBox(children=gas_labels)
dates_vbox = VBox(children=date_labels)
checkbox_vbox = VBox(children=checkbox_objects)

dynamic_box = HBox(children=[site_vbox, gas_vbox, dates_vbox, checkbox_vbox])

download_button = Button(description="Download", button_style="success", layout=table_layout)
download_box = HBox(children=[download_button])

status_bar = HTML(value="Status: Waiting...", layout=statusbar_layout)

complete = VBox(children=[header_box, dynamic_box, download_box, status_bar])

selected_data = []
def select_data(**kwargs):
    selected_data.clear()
    
    for key in kwargs:
        if kwargs[key] is True:
            selected_data.append(key)

def update_statusbar(text):
    status_bar.value = F"Status: {text}"
    
data = None
def download_data(args):
    """ Download the data in the selected keys from the object store
    
        Args:
            ?
        Returns:
            Pandas.Dataframe of selected data
    """
    from HUGS.Client import Retrieve
    from pandas import read_json as pd_read_json
    
    update_statusbar("Downloading...")
    
    download_keys = {key: date_keys[key]["keys"] for key in selected_data}
    
    retrieve = Retrieve(service_url=base_url)
    
    global data
    data = retrieve.retrieve(keys=download_keys)
    
    # Conver the JSON into Dataframes
    for key in data:
        data[key] = pd_read_json(data[key])
    
    # Update the status bar
    if len(data) > 0:
        update_statusbar("Download complete")
    else:
        update_statusbar("No data downloaded")
            
download_button.on_click(download_data)
out = widgets.interactive_output(select_data, arg_dict)

display(complete, out)

Output()

In [ ]:
data.keys()

dict_keys(['bsd_co2', 'hfd_co2', 'bsd_co', 'hfd_co'])

Retrieve the data from the object store

In [ ]:
selected_data

['bsd_co2', 'hfd_co2', 'bsd_co', 'hfd_co']

In [ ]:
data

{'bsd_co2':                      co2 count  co2 stdev  co2 n_meas
 2014-01-30 10:52:30     409.66      0.028          26
 2014-01-30 10:53:30     409.50      0.058          26
 2014-01-30 10:54:30     409.50      0.026          25
 2014-01-30 10:55:30     409.37      0.044          26
 2014-01-30 10:56:30     409.45      0.023          25
 ...                        ...        ...         ...
 2014-01-30 14:16:30     408.82      0.032          26
 2014-01-30 14:17:30     408.76      0.060          25
 2014-01-30 14:18:30     408.79      0.030          26
 2014-01-30 14:19:30     408.80      0.024          26
 2014-01-30 14:20:30     408.78      0.019          25
 
 [76 rows x 3 columns],
 'hfd_co2':                      co2 count  co2 stdev  co2 n_meas
 2013-12-04 14:02:30     414.21      0.109          19
 2013-12-11 12:52:30     410.96      0.258          19
 2013-12-18 19:12:30     402.34      0.091          19
 2013-12-25 22:56:30     401.66      0.317          19
 2014-01-01 18:25

Plot the data we've selected

Select the data from the downloaded data using the checkboxes and then click plot to update the figure

In [ ]:
from bqplot import pyplot as plt
from bqplot import DateScale, LinearScale, Axis, Lines, Figure
from random import randint    
from numpy import random as np_random
import numpy as np

# Create some checkboxes
plot_checkboxes = []
plot_keys = [] 

for key in data:
    # Create a more readable description
    desc = " ".join(key.split("_")).upper()
    plot_keys.append(key)
    plot_checkboxes.append(Checkbox(description=desc, value=False))
    
select_instruction = HTML(value="<b>Select data: </b>", layout=table_layout)
plot_button = Button(description="Plot", button_style="success", layout=table_layout)

select_box = HBox(children=[select_instruction])
checkbox_box = VBox(children=plot_checkboxes)
horiz_select = HBox(children=[select_box, checkbox_box])
plot_box = HBox(children=[plot_button])
ui_box = VBox(children=[horiz_select, plot_box])

arg_dict = {plot_keys[i]: checkbox for i, checkbox in enumerate(plot_checkboxes)}

selected_data = []
def select_data(**kwargs):
    selected_data.clear()
    
    for key in kwargs:
        if kwargs[key] is True:
            selected_data.append(key)
    
    print(selected_data)
    
# Link the checkbox selection with the selected dataframes to plot
out = widgets.interactive_output(select_data, arg_dict)

output = widgets.Output()

def plot_data(arg):
    """ Each key in the data dict is a dataframe
    
    """
    # Here take the keys in the selected data list and use them to
    # access the Dataframes to plot
    # Use the same axes. Can have a button to create new plots etc in the future
    
    # TODO - change this to take the data directly from the dict?
    plot_data = [data[x] for x in selected_data]
    
    # For now just plot the first column in the data
        
     # Setup the axes    
    
    x_scale = DateScale()
    y_scale = LinearScale()
    scales = {"x": x_scale, "y": y_scale}
    
    r = lambda: randint(0,255)
#     lines = [Lines(x=d.index.values, y=d.iloc[:,0], scales=scales, colors=['#%02X%02X%02X' % (r(),r(),r())]) for d in plot_data]
#     lines.x = plot_data[0].index.values
#     fig.marks = [Lines(x=d.index.values, y=d.iloc[:,0], scales=scales, colors=['#%02X%02X%02X' % (r(),r(),r())]) for d in plot_data]
    print(type(plot_data[0].index.values))
    
    lines.x = [d.index.values.tolist() for d in plot_data]    
    lines.y = [d.iloc[:,0] for d in plot_data]
    
    print(plot_data[0].index.values.tolist())
    
#     print(type(plot_data[0].first_valid_index()))
    
#     print(lines.x)
    
    
#     lines.x = np.arange(100)
#     lines.y = np.cumsum(np.random.randn(2, 100), axis=1)
#     lines.x = [Lines(x=d.index.values, y=d.iloc[:,0], scales=scales, colors=['#%02X%02X%02X' % (r(),r(),r())]) for d in plot_data]
    
    
x_scale = DateScale()
y_scale = LinearScale()
scales = {"x": x_scale, "y": y_scale}

ax = Axis(label="Date", scale=x_scale)
# TODO - this could be updated depending on what's being plot
ay = Axis(label="Count", scale=y_scale, orientation="vertical")

lines = Lines(x=np.arange(100), y=np.cumsum(np.random.randn(2, 100), axis=1), scales=scales)
fig = Figure(marks=[lines], axes=[ax, ay], animation_duration=1000)
    
plot_button.on_click(plot_data)

# plot_button.on_click(lambda btn: plot_data())

display(ui_box, out)

VBox([fig])

Output()

<class 'numpy.ndarray'>
[1386165750000000000, 1386766350000000000, 1387393950000000000, 1388012190000000000, 1388600730000000000, 1389189810000000000, 1394585790000000000, 1395194790000000000, 1395801570000000000, 1396407390000000000, 1397013270000000000, 1397633670000000000, 1398232170000000000, 1398827550000000000, 1399422510000000000, 1400033550000000000, 1400627370000000000, 1401215910000000000, 1401803250000000000, 1404834150000000000, 1405421190000000000, 1406618010000000000, 1407206070000000000, 1407794610000000000, 1408402890000000000, 1408991430000000000, 1409578770000000000, 1410177090000000000, 1410779670000000000, 1411397970000000000, 1411991370000000000, 1412580270000000000, 1413223830000000000, 1414520430000000000, 1415163510000000000, 1415805510000000000, 1416461910000000000, 1417102110000000000, 1417753110000000000, 1418387910000000000, 1419113550000000000, 1419753750000000000, 1420392150000000000, 1421032350000000000, 1421690550000000000, 1422328950000000000, 142297095

In [ ]:
fig.animation_duration

750

In [ ]:
from bqplot.scales import ColorScale
print(ColorScale.colors)

In [ ]:
plot_checkboxes

Now we can plot some of the data

In [ ]:
import pandas as pd
# co2_data = pd.read_json(data["results"]["bsd_co2"])
co_data = pd.read_json(data["results"]["bsd_ch4"])


In [ ]:
co_data.head(10)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

fig = plt.figure()
# ax = fig.add_subplot(211)
# ax.plot(co2_data.index.values, co2_data["co2 count"], color = "#4e79a7" )
ax = fig.add_subplot(111)
ax.plot(co_data.index.values, co_data["ch4 count"], color = "#59a14f")

plt.show()

In [ ]:
# Checkboxes for selecting which data we want to plot
# Select the gas, it goes and plots the 
# Simple line plot

In [ ]:
from ipyleaflet import (
    Map,
    Marker, MarkerCluster, TileLayer, ImageOverlay, GeoJSON,
    Polyline, Polygon, Rectangle, Circle, CircleMarker, Popup,
    SplitMapControl, WidgetControl,
    basemaps, basemap_to_tiles
)

from ipywidgets import HTML

In [ ]:
center = [54.2361, -4.548]
zoom = 5
m = Map(center=center, zoom=zoom)
mark_bsd = Marker(location=(54.942544, -1.369204))
mark_bsd.popup = HTML(value="Bilsdale")

mark_mhd = Marker(location=(53.20,-9.54))
mark_mhd.popup = HTML(value="Macehead")

mark_tac = Marker(location=(52.511, 1.155003))
mark_tac.popup = HTML(value="Tacolneston")

m += mark_bsd
m += mark_mhd
m += mark_tac

display(m)

In [ ]:
m